# 第三章 哈希算法

哈希算法又称散列函数算法，是一种查找算法。简单来说，就是把一些复杂的数据，通过某种函数映射关系，映射成更加易于查找的方式。但是这种映射关系有可能会发生多个关键字映射到同一地址的现象，我们称之为冲突。在这种特殊情况下，需要对关键字进行二次或者更多次的处理，在其他大多数情况下，哈希算法可以在常数时间内储存和查找这些关键字。

本章主要设计的知识点和问题有如下几项。
- 什么是哈希：简单介绍哈希及哈希的原理
- 两个数的和：快速寻找两个数的和
- 单词模式匹配：简单的模式匹配问题

## 3.1 什么是哈希

常见的数据查找算法一般可以分为以下几种：
- 顺序查找
- 二分查找
- 深度优先遍历
- 广度优先遍历
- 哈希查找

- 顺序查找是最简单的方式，需要对数据集中的数据逐个进行匹配，所以效率相对低，不太适合大数据量的查找问题。
- 二分查找的效率高，但是要求数据必须有序，而对数据排序通常需要更多的时间开销。
- 深度优先遍历和广度优先遍历对于大数据量的查找问题效率并不高。
- 哈希查找由于其查找速度快，查询、插入和删除操作简单等原因，获得了广泛的应用。很多问题本质上都是查找问题。解决查找问题，哈希算法是较好的选择。

一般情况下，哈希算法的查询效率可以达到常数级别，哈希表成为直接寻址的有效替代方案。然而，由于关键字的取值可能在一个很大的范围，数据在通过哈希函数进行映射的时候，很难找到一个哈希函数，使得这些关键字不能映射到唯一的值，就会出现多个关键字映射到同一个值的现象，这种现象我们成为冲突。

解决哈希冲突的方法有很多，比如链地址法、二次再散列法、线性探测再散列法、建立一个公共溢出区等方法。

本章主要介绍哈希算法的应用，因此只对链地址法进行讲解。链地址法处理冲突的方法本质上是一种数组加链表的方法，当发生多个数据通过哈希函数映射得到相同的哈希值时，通常把具有相同哈希地址的关键字放在同一个链表中，称该链表为同义词链表，该链表也被称为桶。

链地址法的储存过程是这样的，首先建立一个数组Hash储存所有链表的头指针，由数据的关键字Key通过对应的哈希函数计算出哈希地址，找到相应的桶号，之后建立新的节点储存改数据，并把节点放到桶内链表的最后面或者最前面。

和储存的方法类似，查找数据的时候，也是有数据的关键字通过哈希函数计算关键字对应的哈希地址，之后顺序比较桶的内部节点是否与所查找的关键字一样直到找到数据为止。如果全部的节点都不和关键字一样，则说明哈希表里面没有该数据。这个解决冲突的方法对哈希函数的要求很高，如果哈希函数选得不太好的话，哈希表的查找效率会退化为链表的查找，也就是顺序查找。

用链地址法构造的散列表，插入和删除节点操作易于实现，所以构造链表的时间开销很低，但是指针需要额外的地址空间，当数据量很大的时候，会扩大哈希表规模，内存空间需求较大。

从上面的分析可以看到，构造优秀的哈希函数和选择解决冲突的方法是哈希查找算法的关键，不管多么高明的算法都不肯能避免冲突问题，但是构建计算简单、高效快速，并且能够将关键字集合均匀地分布在地址集中，使得冲突达到最小，一直是计算机科学家追求的目标。

## 3.2 两个数的和

### 3.2.1 问题求解1

给定五个数字：3,4,5,7,10，从中选出两个数使得他们的和为11，可以选择4和7。我们可以使用双指针法去解决这个问题。首先对数据进行排序，然后建立两个指针Left和Right，分别指向新数组的首末元素。如果两个指针指向的两个数据相加的和等于目标值，那么查找结束，返回这两个数的下标；如果它们的和小于目标值，则说明Left指向的数据太小了，需要让Left向右移动一位；如果他们的和大于目标值，这说明Right指向的数据太大了，需要让right指针向左移动一位。重复这个过程，知道找到答案。




In [12]:
def twoSum(nums, target):
    res = [] 
    newnums = nums[:]
    newnums.sort()
    left = 0
    right = len(newnums) - 1
    while left < right:
        if newnums[left] + newnums[right] == target:
            for i in range(len(nums)):
                if nums[i] == newnums[left]:
                    res.append(i)
                    break
            for i in range(len(nums)-1,-1,-1):
                if nums[i] == newnums[right]:
                    res.append(i)
                    break
            res.sort()
            break
        elif newnums[left] + newnums[right] > target:
            right -= 1
        else :
            left += 1
#     if  len(res) == 0: 
    if not res:
        res = [-2,-2]
    return(res[0]+1,res[1]+1)
    

In [13]:
twoSum(nums= [1,2,5,6,9,10],target=100)

(-1, -1)

### 3.2.3 问题求解2

在第一种方法的基础上，我们需要更加高效的算法，下面会介绍哈希函数是如何帮助解决这个问题的。

首先，研究一下在双指针问题中，时间被浪费在了哪里？双指针算法的前提是数据有序，所以解法1的第一步是对数据进行排序，即使是使用最优秀的排序算法也是比较消耗时间的。接下来，为了寻求答案，左指针和右指针不断移动，当左右指针发现了答案的时候，由于需要返回原始数据的位置，还要去原始数组中寻找这两个数据的位置。这两个操作都是比较耗时的，并且没有必要。

为了使用哈希算法，我们需要首先建立一个字典。然后每当给定一个数m，都去字典中查询是否有一个数等于target - m。另外，可以通过字典记录目前已经出现过的数字，这样每次出现一个新的数字的时候，就去字典中查找有没有对应的数字，如果有就说明找到了，没有的话就把该数字放到字典中去，方便之后的查询。

In [14]:
def twoSum(nums, target):
    dict = {}
    for i in range(len(nums)):
        m = nums[i]
        if m in dict:
            continue
        elif target - m in dict:
            return(dict[target - m] + 1, i + 1)
        dict[m] = i

In [18]:
twoSum(nums= [1,2,5,6,9,10],target=10)

(1, 5)

In [53]:
nums = [1,2,5,6,9,10]

In [66]:
def twoSum(nums, target):
    dict1 = {}
    for i in range(len(nums)):
        m = nums[i]
        if target - m in dict1:
            return(dict[target - m] + 1, i + 1)
        dict1[m] = i

In [79]:
%timeit twoSum(nums= [1,2,5,6,9,10],target=10)

778 ns ± 4.37 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [81]:
def twoSum2(nums, target):
    dict1 = {j:i for i,j in enumerate(nums)}
    for i in range(len(nums)):
        m = nums[i]
        if target - m in dict1:
            return(dict1[target - m] + 1, i + 1)
        

In [82]:
%timeit twoSum2(nums= [1,2,5,6,9,10],target=10)

946 ns ± 7.72 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


从以上两种方法可以看出，匹配两个数的和是否等于target，需要两个数。所以如果一个数没能匹配到，再把它加入字典，等到后续有值能和字典中的值相加等于target的时候，自然会终止。并不需要一开始就生成所有数字的字典，会浪费时间。通过列表生成式生成的时间和通过循环的时间差不多。

In [84]:
%timeit [i*2+1 for i in range(10000)]

607 µs ± 3.73 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [1]:
%%timeit 
list_1 = []
for i in range(10000):
    list_1.append(i*2+1)

825 µs ± 5.24 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 3.3 单词模式匹配

模式匹配是一个经典的问题，。首先给定两个字符串，一个是单词模式字符串，一个是目标字符串。之后检查目标字符串是否为给定的单词模式，及求目标字符串中单词出现的规律是否符合单词模式字符串中的规律。

例如，单词模式字符串为“一二二一”：
目标字符串为“苹果 香蕉 香蕉 苹果” ，则规律一样，匹配成功。
目标字符串为“香蕉 苹果 香蕉 苹果”，则规律不一样，匹配不成功。
目标字符串为“香蕉 香蕉 香蕉 香蕉”，则规律不一样，匹配不成功。

### 3.3.1 问题求解

如果单词模式字符串和目标字符串只存在一对一的对应关系，不存在一对多和多对多的对应关系，就可以说明两个字符串匹配成功。

现在的问题转化为寻找映射关系，本质上也是一个查找问题。既然是查找问题，第一个想到的就是能否使用哈希算法。

对于本题来说，需要建立模式字符串中每个字符和目标字符串中每个单词之间的映射关系。而哈希表本身就是一种映射关系，因此可以用哈希表储存这种关系。


### 3.3.2 最终代码


In [56]:
def wordPattern(wordPattern, input):
    word = input.split(' ')
    if len(word) != len(wordPattern):
        return False
    hash_dict = {}
    used_dict = {}
    for i in range(len(wordPattern)):
        print("正在检测第{0}个位置".format(i))
        if wordPattern[i] in hash_dict:
            if hash_dict[wordPattern[i]] != word[i]:
                print("第{0}个位置错误,错误原因是模式字符串的值,映射为两个目标字符串的值".
                      format(i))
                return False
        else:
            if word[i] in used_dict:
                print("第{0}个位置错误，错误原因是模式字符串的值未出现过，但是该位置的目标字符串已经使用过".
                      format(i))
                return False
        hash_dict[wordPattern[i]] = word[i]
        used_dict[word[i]] = True
    return True
            

In [57]:
wordPattern('1112','苹果 苹果 香蕉 香蕉')
wordPattern('1122','苹果 苹果 苹果 香蕉')

正在检测第0个位置
正在检测第1个位置
正在检测第2个位置
第2个位置错误,错误原因是模式字符串的值,映射为两个目标字符串的值
正在检测第0个位置
正在检测第1个位置
正在检测第2个位置
第2个位置错误，错误原因是模式字符串的值未出现过，但是该位置的目标字符串已经使用过


False

## 3.4 猜词游戏

猜词游戏Bulls and Cows。游戏的玩法是这样的，一个人写下几个数字让另外一个人猜，每次答题方猜完之后，出题方给一个提示，告诉他刚才猜测中，有多少位数字和确切位置猜对了（Bulls，公牛），还有多少位数字猜对了但是位置不对（Cows，奶牛）。答题方根据提示继续猜测。

可以保证的是，秘密数字和猜测数字的位数是一样的。我们写一个程序，用A表示公牛，B表示奶牛。

例如秘密数字的2018，猜测数字是8021，0的位置和秘密数字一致，所以算作Bulls，其余三个数字都猜对了，但是位置不对，算作是Cows。我们的程序应该返回1A3B。

例如秘密数字是1123，猜测数字的9111。9111中的第一个1，数字和位置都对，算是Bulls。其他两个1，数字猜对了，但是位置不对，并且只能算是一个奶牛，因为一旦秘密梳子中的某一位和猜测中的某一位匹配，那么该数字就不能和其他数字匹配了。我们的程序应该返回1A1B。

In [75]:
def getHint(secret, guess):
    """
    :type secret:int
    :type guess:int
    :rtype: int
    """
    secret = str(secret)
    guess = str(guess)
    secret_dict = {}
    guess_dict = {}
    
    A = 0
    B = 0
    for i in range(len(secret)):
        if secret[i] == guess[i]:
            A += 1
        else :
            if secret[i] in secret_dict:
                secret_dict[secret[i]] = secret_dict[secret[i]] + 1
            else:
                secret_dict[secret[i]] = 1
            if guess[i] in guess_dict:
                guess_dict[guess[i]] = guess_dict[guess[i]] + 1
            else:
                guess_dict[guess[i]] = 1
    for digit in secret_dict:
        if digit in guess_dict:
            B += min(secret_dict[digit], guess_dict[digit])
    return str(A) + "A" + str(B) + "B"

In [77]:
getHint(1234,1111)

'1A0B'

# 3.5 神奇的词根

游戏的玩法是这样的，给定一个由很多词根组成的字典和一个句子。需要将句子中的所有继承用词都用词跟给替换掉。如果继承词中有许多形成它的词根，则用最短的词根替换它。

例如，字典为["cat","bat","rat"]，句子为"the cattle was rattled by the battery"，经过替换，输出句子为"the cat was rat by the bat" 

### 3.5.1 暴力破解法

对于暴力破解法，由于输入的是一个句子，所以需要先把句子转换为列表。

对于词典中的每一个词根来说，一次判断句子中的每一个单词是否以这个词根开头，如果以它开头的话，就把该单词用词根替换掉。假如该词根的位数为n，可以通过截取第i个单词的前n位，来进行比较判断。

然后再变脸所有的词根，对每个词根进行相同的操作，这样最短的词根替换结果会被保留。最后用join函数进行句子单词的拼接。


In [80]:
def replaceWords(dict, sentence):
    """
    :type dict: List[str]
    :type sentence: str
    :rtype: str
    """
    
    s = sentense.split(" ")
    for item in dict:
        for i in range(len(s)):
            n = len(item)
            if item == s[i][:n]:
                s[i] = item
    return " ".join(s)

在暴力破解法中，对于每一个词根，需要在所有的单词中截取相应位数的字字符串，假如词根数量很多或者单词数量很多，双重循环的效率会比较低。在这个过程中，可以把用单词去匹配词根的过程通过哈希来优化。

首先建立两个字典，并做好初始化工作。在建立字典的时候可以使用，collections.defaultdict()建立一个默认字典，它本身是一个字典，只不过Python会自动为它的键赋一个初始值。

In [140]:
def replaceWords(root_dict, sentense):
    """
    :type dict: List[str]
    :type sentense: str
    :rtype: str
    """
    import collections
    d = collections.defaultdict(set)
    s = collections.defaultdict(int)
    sentense = sentense.split()
    for w in root_dict:
        print(w[0])
        d[w[0]].add(w)
        s[w[0]] = max(s[w[0]],len(w))
    for i, w in enumerate(sentense):
        for j in range(s[w[0]]):
            if w[:j+1] in d[w[0]]: ### j从0开始，所以这边要+1
                sentense[i] = w[:j+1]
                break
    return " ".join(sentense)

In [141]:
replaceWords(root_dict=["cat","bat","rat"],sentense="the cattle was rattled by the battery")

c
b
r


'the cat was rat by the bat'

[1, 2]